In [3]:
import pandas as pd
import json
import streamlit as st
import plotly.express as px

# Load the data from the JSON file
with open('fertility_gdp_2014_2024.json') as f:
    data = json.load(f)

# Convert the JSON data into a DataFrame
df = pd.DataFrame(data)


In [2]:
# Pivot the data so that each row contains both Fertility Rate and GDP for the same year
df_pivot = df.pivot_table(index=['Country Name', 'Country Code', 'Year'], 
                          columns='Series Name_y', 
                          values='GDP', 
                          aggfunc='first').reset_index()

# Flatten the columns to remove multi-level index
df_pivot.columns.name = None

# Rename columns for easier understanding
df_pivot = df_pivot.rename(columns={
    'GDP per capita (current US$)': 'GDP_per_Capita',
    'GDP per capita growth (annual %)': 'GDP_Growth_Percent'
})

# Also, merge the fertility rate column (which is under "Series Name_x")
fertility_data = df[df['Series Name_x'] == 'Fertility rate. total (births per woman)']
fertility_data = fertility_data[['Country Name', 'Country Code', 'Year', 'Fertility Rate']]

# Merge fertility rate with GDP data
df_merged = pd.merge(df_pivot, fertility_data, on=['Country Name', 'Country Code', 'Year'])

# Check the first few rows to ensure the data is correctly merged
df_merged.head()


,Country Name,Country Code,Year,GDP_per_Capita,GDP_Growth_Percent,Fertility Rate
0,Afghanistan,AFG,2014,626.51,-0.96,5.56
1,Afghanistan,AFG,2014,626.51,-0.96,5.56
2,Afghanistan,AFG,2015,566.88,-1.67,5.41
3,Afghanistan,AFG,2015,566.88,-1.67,5.41
4,Afghanistan,AFG,2016,523.05,-0.35,5.26


In [6]:
# Ensure GDP Growth Percent is non-negative
df_merged['GDP_Growth_Percent_Abs'] = df_merged['GDP_Growth_Percent'].abs()

# Scatter Plot - Fertility vs GDP
fig = px.scatter(df_merged, x='GDP_per_Capita', y='Fertility Rate', color='Country Code',
                 size='GDP_Growth_Percent_Abs', hover_name='Country Name', 
                 title="Fertility Rate vs GDP Per Capita")
st.plotly_chart(fig)


ValueError: 
    Invalid element(s) received for the 'size' property of scattergl.marker
        Invalid elements include: [nan, nan, nan, nan, nan, nan, nan, nan, nan]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above